In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.central_ids import IDS
from src.utils import cal_metric, print_results
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score, confusion_matrix
import pickle
import torch
from flwr.common.parameter import parameters_to_ndarrays
from collections import OrderedDict
from utils import set_parameters, test_model

from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
car_model = 'Kia'
args = {
    'data_dir': f'../../Data/LISA/{car_model}/1/',
    'save_dir': f'../save/FedAvg_FN/{car_model}/',
    'C': 2,
    'B': 128,
    'epochs': 5,
    'val_freq': 1,
    'lr': 1e-3,
    'weight_decay': 1e-4
}

# Fine tuning the federated model

In [4]:
rnd = 40
ckpt_file = f'../save/federated/ckpt-rnd{rnd}.npz'
tensors = np.load(ckpt_file, allow_pickle=True)['arr_0'][()]
data_dir = f'../../Data/LISA/{car_model}/1/'
model = IDS(**args)

In [5]:
weights = parameters_to_ndarrays(tensors)
params_dict = zip(model.state_dict().keys(), weights)
state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
model.load_state_dict(state_dict, strict=True)

<All keys matched successfully>

In [6]:
for name, param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [7]:
logger = loggers.TensorBoardLogger(save_dir=args['save_dir'],) 
checkpoint_callback = ModelCheckpoint(
        filename='{epoch:02d}-{val_loss:.2f}-{val_f1:.4f}',
        every_n_epochs=args['val_freq'],
        monitor='val_loss', 
        save_top_k=5)

In [8]:
trainer = pl.Trainer(max_epochs=args['epochs'], accelerator='cuda', 
                        gpus=[0], logger=logger, log_every_n_steps=100, 
                        check_val_every_n_epoch=args['val_freq'],
                        callbacks=[checkpoint_callback])

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model)

Missing logger folder: ../save/FedAvg_FN/Tesla/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Classifier       | 700 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
699 K     Trainable params
258       Non-trainable params
700 K     Total params
2.801     Total estimated model params size (MB)


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

Sanity Checking: 0it [00:00, ?it/s]

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Validation f1 score: 0.9102


...

Validation: 0it [00:00, ?it/s]

Validation f1 score: 0.9985


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Validation f1 score: 0.9984


## Run model from checkpoint

In [4]:
ckpt_files = '../save/FedAvg_FN/Kia/lightning_logs/version_1/checkpoints/epoch=04-val_loss=0.00-val_f1=0.9994.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
test_model(args['data_dir'], fedavg_model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9993265993265994, 0.00042444821731748726)

In [4]:
ckpt_files = '../save/Kia/lightning_logs/version_1/checkpoints/epoch=49-val_loss=0.00-val_f1=0.9992.ckpt'
local_model = IDS.load_from_checkpoint(ckpt_files, C=2)
test_model(args['data_dir'], local_model)

NotImplementedError: Could not run 'aten::empty_strided' with arguments from the 'MPS' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::empty_strided' is only available for these backends: [Dense, FPGA, Conjugate, Negative, ZeroTensor, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseXPU, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN